# US Visa Applicants 2012-2017



### Hypothesis 1: 

If applicants are from a Fortune 500 company, it is more probable that their visa will be certified.


### Hypothesis 2: 

There is a relationship between annual salaries offered to candidates and their country of origin in the technology industry for software developer roles. 


### Hypothesis 3: 

There is a difference in certification rates between US ECONOMIC STEM SECTORS and NON_STEM SECTORS.


### Hypothesis 4: 

There is a difference in visa certification rates between candidates offered salaried (yearly paid) positions and those offered hourly positions.



#### Data from Kaggle : https://www.kaggle.com/jboysen/us-perm-visas

In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import statsmodels.api as sm
from statsmodels.formula.api import ols
import scipy.stats as stats
import math
from operator import itemgetter
import warnings
warnings.filterwarnings('ignore')
import re

## Initial Data Exploration

In [78]:
df = pd.read_csv('us_perm_visas.csv')

In [79]:
df.head(2)

,add_these_pw_job_title_9089,agent_city,agent_firm_name,agent_state,application_type,case_no,case_number,case_received_date,case_status,class_of_admission,...,ri_pvt_employment_firm_to,ri_us_workers_considered,schd_a_sheepherder,us_economic_sector,wage_offer_from_9089,wage_offer_to_9089,wage_offer_unit_of_pay_9089,wage_offered_from_9089,wage_offered_to_9089,wage_offered_unit_of_pay_9089
0,NaN,NaN,NaN,NaN,PERM,A-07323-97014,NaN,NaN,Certified,J-1,...,NaN,NaN,NaN,IT,75629,NaN,yr,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,PERM,A-07332-99439,NaN,NaN,Denied,B-2,...,NaN,NaN,NaN,Other Economic Sector,37024,NaN,yr,NaN,NaN,NaN


In [111]:
#list(df.columns)

In [80]:
df.shape

(374362, 154)

In [81]:
#list(df.columns)

In [82]:
a_df = df[['pw_soc_code','pw_soc_title','case_status', 'class_of_admission', 'country_of_citizenship','pw_amount_9089','pw_level_9089', 'decision_date','employer_name','employer_num_employees','employer_city']]

In [83]:
sal_chk = a_df.loc[a_df['pw_soc_title']=='Software Developers, Applications']

In [84]:
#sal_chk[['pw_level_9089','pw_amount_9089']]

In [85]:
cert_df = a_df[a_df['case_status']=='Certified']

In [86]:
cert_titles = cert_df.groupby(['pw_soc_title'])['pw_soc_code'].count().sort_values(ascending=False)

In [87]:
cert_titles.head()

pw_soc_title
Software Developers, Applications            57998
Computer Systems Analysts                    17040
Software Developers, Systems Software        11329
Electronics Engineers, Except Computer        6591
Computer and Information Systems Managers     4569
Name: pw_soc_code, dtype: int64

In [88]:
class_cert = cert_df.groupby(['class_of_admission'])['pw_soc_title'].count().sort_values(ascending=False)
class_cert.head(8)

class_of_admission
H-1B          139297
L-1             9946
F-1             7391
Not in USA      4190
TN              2194
E-2             1973
B-2             1316
Parolee         1052
Name: pw_soc_title, dtype: int64

In [89]:
cert_cities = cert_df.groupby(['employer_city'])['pw_soc_code'].count().sort_values(ascending=False)
cert_cities.head()

employer_city
NEW YORK           8445
SANTA CLARA        5943
COLLEGE STATION    5661
MOUNTAIN VIEW      4597
SAN JOSE           4466
Name: pw_soc_code, dtype: int64

In [90]:
cert_employer = cert_df.groupby(['employer_name'])['pw_soc_code'].count().sort_values(ascending=False)
cert_employer.head()

employer_name
COGNIZANT TECHNOLOGY SOLUTIONS US CORPORATION    5607
INTEL CORPORATION                                3658
MICROSOFT CORPORATION                            3253
GOOGLE INC.                                      2972
AMAZON CORPORATE LLC                             2104
Name: pw_soc_code, dtype: int64

In [91]:
b_df = df[['pw_soc_code','pw_soc_title','case_status', 'class_of_admission', 'country_of_citizenship','pw_amount_9089', 'decision_date','employer_name','employer_num_employees','employer_city']]
no_cert_df = b_df[b_df['case_status']!='Certified']
no_cert_titles = no_cert_df.groupby(['pw_soc_title'])['pw_soc_code'].count().sort_values(ascending=False)

In [92]:
no_cert_titles.head()

pw_soc_title
Software Developers, Applications            56843
Computer Systems Analysts                    19660
Software Developers, Systems Software        10027
Electronics Engineers, Except Computer        6601
Computer and Information Systems Managers     5062
Name: pw_soc_code, dtype: int64

In [93]:
class_no_cert = no_cert_df.groupby(['class_of_admission'])['pw_soc_title'].count().sort_values(ascending=False)
class_no_cert.head()

class_of_admission
H-1B          142559
L-1             9938
F-1             7491
Not in USA      4383
E-2             2246
Name: pw_soc_title, dtype: int64

In [94]:
no_cert_cities = no_cert_df.groupby(['employer_city'])['pw_soc_code'].count().sort_values(ascending=False)
no_cert_cities.head()

employer_city
NEW YORK           8281
COLLEGE STATION    6457
REDMOND            5373
SANTA CLARA        5170
SAN JOSE           4881
Name: pw_soc_code, dtype: int64

In [95]:
no_cert_employer = no_cert_df.groupby(['employer_name'])['pw_soc_code'].count().sort_values(ascending=False)
no_cert_employer.head()

employer_name
COGNIZANT TECHNOLOGY SOLUTIONS US CORPORATION    6474
MICROSOFT CORPORATION                            5116
INTEL CORPORATION                                2906
GOOGLE INC.                                      2193
CISCO SYSTEMS, INC.                              1634
Name: pw_soc_code, dtype: int64

## Data Cleaning

In [96]:
df.country_of_citizenship = df.country_of_citizenship.fillna('')
df.country_of_citzenship = df.country_of_citzenship.fillna('')

In [97]:
df['citizenship'] = df.country_of_citizenship.astype(str)+df.country_of_citzenship.astype(str)
df['citizenship'].isna().value_counts()

False    374362
Name: citizenship, dtype: int64

In [98]:
df.drop(columns=['country_of_citizenship','country_of_citzenship'], inplace=True)
df.drop(columns=['add_these_pw_job_title_9089','agent_city','agent_firm_name','agent_state'], inplace=True)

In [99]:
df.case_no = df.case_no.fillna('')
df.case_number = df.case_number.fillna('')

In [100]:
df['case_num'] = df.case_no.astype(str) + df.case_number.astype(str)

In [101]:
df['case_num'].isna().value_counts()

False    374362
Name: case_num, dtype: int64

In [102]:
df.drop(columns=['case_no','case_number'], inplace=True)

In [103]:
def combine_columns(new_column_name, column1, column2):
    column1 = column1.fillna('')
    column2 = column2.fillna('')
    df[new_column_name] = column1.astype(str) + column2.astype(str)   
    return df

In [109]:
def convert_df(df):
    df = combine_columns('foreign_work_alt_edu', df.foreign_worker_info_alt_edu_experience, df.fw_info_alt_edu_experience)
    df = combine_columns('foreign_worker_birth_country', df.foreign_worker_info_birth_country, df.fw_info_birth_country)
    df = combine_columns('foreign_worker_edu_other', df.foreign_worker_info_education_other, df.fw_info_education_other)
    df = combine_columns('foreign_worker_rel_occ_exp', df.foreign_worker_info_rel_occup_exp, df.fw_info_rel_occup_exp)
    df = combine_columns('foreign_worker_req_exp',  df.foreign_worker_info_req_experience, df.fw_info_req_experience)
    df = combine_columns('naics_codes', df.naics_code, df.naics_us_code)
    df = combine_columns('naics_num', df.naics_codes, df.naics_2007_us_code)
    df = combine_columns('naics_2007', df.naics_2007_us_title, df.naics_us_title_2007)
    df = combine_columns('naics_t', df.naics_title, df.naics_us_title)
    df = combine_columns('naics_title', df.naics_t, df.naics_2007)
    df = combine_columns('wage_offer_from',df.wage_offer_from_9089,df.wage_offered_from_9089)
    df = combine_columns('wage_offer_to',df.wage_offer_to_9089,df.wage_offered_to_9089)
    df = combine_columns('wage_unit_of_pay_offer',df.wage_offer_unit_of_pay_9089,df.wage_offered_unit_of_pay_9089)
    df = combine_columns('pw_job_title', df.pw_job_title_908, df.pw_job_title_9089)
    return df


In [110]:
df = convert_df(df)

In [112]:
df = df[['application_type',
 'case_received_date',
 'case_status',
 'class_of_admission',
 'decision_date',
 'employer_city',
 'employer_country',
 'employer_name',
 'employer_num_employees',
 'employer_state',
 'foreign_worker_info_city',
 'foreign_worker_info_education',
 'foreign_worker_info_state',
 'pw_amount_9089',
 'pw_level_9089',
 'pw_soc_code',
 'pw_soc_title',
 'refile',
 'us_economic_sector',
 'citizenship',
 'case_num',
 'foreign_work_alt_edu',
 'foreign_worker_birth_country',
 'foreign_worker_edu_other',
 'foreign_worker_rel_occ_exp',
 'foreign_worker_req_exp',
 'naics_num',
 'naics_title',
 'wage_offer_from',
 'wage_offer_to',
 'wage_unit_of_pay_offer',
 'pw_job_title']]

In [114]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 374362 entries, 0 to 374361
Data columns (total 32 columns):
application_type                 135269 non-null object
case_received_date               239091 non-null object
case_status                      374362 non-null object
class_of_admission               351517 non-null object
decision_date                    374362 non-null object
employer_city                    374348 non-null object
employer_country                 239019 non-null object
employer_name                    374350 non-null object
employer_num_employees           239013 non-null float64
employer_state                   374320 non-null object
foreign_worker_info_city         239065 non-null object
foreign_worker_info_education    239053 non-null object
foreign_worker_info_state        221898 non-null object
pw_amount_9089                   372146 non-null object
pw_level_9089                    346735 non-null object
pw_soc_code                      373965 non-null

In [115]:
df.employer_city = df.employer_city.str.upper()
df.employer_name = df.employer_name.str.upper()
df.naics_title = df.naics_title.str.upper()
df.pw_job_title = df.pw_job_title.str.upper()
df.us_economic_sector = df.us_economic_sector.str.upper()
df.pw_soc_title  = df.pw_soc_title.str.upper()

In [116]:
df['wage_unit_of_pay_offer'] = df['wage_unit_of_pay_offer'].replace('bi',"Bi-Weekly")
df['wage_unit_of_pay_offer'] = df['wage_unit_of_pay_offer'].replace('mth',"Month")
df['wage_unit_of_pay_offer'] = df['wage_unit_of_pay_offer'].replace('wk',"Week")
df['wage_unit_of_pay_offer'] = df['wage_unit_of_pay_offer'].replace('hr',"Hour")
df['wage_unit_of_pay_offer'] = df['wage_unit_of_pay_offer'].replace('yr',"Year")

In [117]:
df = df.replace("",np.NaN)

In [ ]:
#df.to_csv(r'~/Code/mod_3_project/visa_clean.csv')